In [1]:
#パッケージ読み込み
import re
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup
import unicodedata
import itertools
import string

alpha=string.ascii_letters
p = re.compile(r"<[^>]*?>")

In [2]:
#データ取得

#1次元リスト化
reco_li = []
for i in range(1,62):
    rl = [f"回数>>{i}"]
    url = f"https://jacompa.or.jp/reco{i}.html"
    html = requests.get(url)
    html.encoding = html.apparent_encoding #文字化け防止の行
    html = html.text #テキスト表示
    soup = BeautifulSoup(str(html), 'html.parser') #きれいに
    
    flag = 0
    for j in soup.find_all(['h3', 'p', 'div']):
        text = str(j.get_text(strip=True))
        if flag == 1 and "日本作曲家協会" not in str(text):
            if j.name == "h3" and "賞" in text and " 日本レコード大賞" not in text:
                text = f"<<{text}>>"
                rl.append(text)
            elif j.name == "p" and "style=" not in str(j) and "主催：" not in text and text != "" and "後援：" not in text:
                rl.append(text)
            elif j.name == "div" and '曲名：' in text and "[" not in text and "大賞曲名" not in text:
                rl.append(text)
            elif i >= 58 and j.name == "div" and "c-body c-center" in str(j) and text != "" and 'www.jacompa.or.jp' not in text and "50音順" not in text:
                rl.append(text)
        #if j.name == "p" and "主催：" in text:
        if j.name == "div" and "column -column1" in str(j):
            flag = 1
    r = "".join(rl)
    reco_li.append(r)

In [3]:
#多重リスト化
for i in range(len(reco_li)):
    reco_li[i] = reco_li[i].replace("\u3000", " ")
    reco_li[i] = reco_li[i].split("<<")
    for j in range(len(reco_li[i])):
        reco_li[i][j] = reco_li[i][j].split(">>")

In [12]:
#辞書化
li_all = []
for i in range(len(reco_li)):
    li_d = {}
    for j in range(len(reco_li[i])):
        if reco_li[i][j][0] in li_d:
            li_d[reco_li[i][j][0]] = li_d[reco_li[i][j][0]]+reco_li[i][j][1]
        else:
            li_d[reco_li[i][j][0]] = reco_li[i][j][1]
    li_all.append(li_d)
li_all[32]

#データフレーム化
col = []
ele = []
for k in li_all[0].keys():
    k = k.lstrip("【").rstrip("】")
    col.append(k)
for v in li_all[0].values():
    ele.append(v)
df_1 = pd.DataFrame(ele, index = col).T
df_1
for i in range(len(li_all)-1):
    col = []
    ele = []
    for k in li_all[i+1].keys():
        k = k.lstrip("【").rstrip("】")
        col.append(k)
    for v in li_all[i+1].values():
        ele.append(v)
    df = pd.DataFrame(ele, index = col).T
    df_1 = pd.concat([df_1,df])
df_1.to_csv("df_o.csv", index = False)

In [13]:
df = pd.read_csv("df_o.csv")

#いらない列消す
delete_li = ["作曲", "作詩", "企画賞", "編曲賞", "特別賞", "功労賞",
             "中山晋平賞", '西条八十賞', "最優秀スター", "ベストアーティスト",
             "特別選奨", "音楽文化","アジア","文化賞"
             '審査委員', "TBS", "古賀政男","顕彰", "表彰",
             "日本作曲家協会", "アルバム", "特別映画音楽", "特別歌謡音楽",
             "特別大衆", "ゴールデン・アイドル", "吉田正", "美空ひばり","外国", "周年記念賞",
            ]

d_num = []
for i in range(len(df.columns.values)):
    if df.columns.values[i] == "作品賞": #なぜか作品賞というのがあった
        d_num.append(i)
    for d in delete_li:
        if d in df.columns.values[i]:
            d_num.append(i)
            break

df = df.drop(columns=df.columns[d_num])
df.to_csv("df_official.csv", index = False)

In [14]:
#やるならここから
df_recotai_all = pd.read_csv("df_official.csv")
#df_recotai_all

songs = []
artis = []
genre = []
year = []

for i in range(len(df_recotai_all)):
    s = []
    a = []
    y = []
    g = []
    for j in range(len(df_recotai_all.iloc[i])):
        if not pd.isnull(df_recotai_all.iloc[i][j]):
            t = str(df_recotai_all.iloc[i][j])
            if "「" in t or "『" in t: #曲名が含まれてるかどうか
                if "曲名：「" in t: #「曲名：」表記がされてるかどうか
                    if t.startswith("曲名：「"): #「曲名：」で始まるかどうか
                        t = t.lstrip("曲名：「")
                        t = t.split("曲名：「")
                        for k in range(len(t)):
                            t[k] = t[k].split("」")
                            s.append(t[k][0]) #曲名部分
                            a.append(t[k][1]) #アーティスト部分
                            g.append(df_recotai_all.columns.values[j])
                            y.append(i+1)
                    else:
                        t = t.rstrip("」")
                        t = t.split("」")
                        for k in range(len(t)):
                            t[k] = t[k].split("曲名：「")
                            s.append(t[k][1]) #曲名
                            a.append(t[k][0]) #アーティスト
                            g.append(df_recotai_all.columns.values[j])
                            y.append(i+1)
                elif "曲名：『" in t: #「曲名：」表記がされてるかどうか
                    if t.startswith("曲名：『"): #「曲名：」で始まるかどうか
                        t = t.lstrip("曲名：『")
                        t = t.split("曲名：『")
                        for k in range(len(t)):
                            t[k] = t[k].split("』")
                            s.append(t[k][0]) #曲名部分
                            a.append(t[k][1]) #アーティスト部分
                            g.append(df_recotai_all.columns.values[j])
                            y.append(i+1)
                    else:
                        t = t.rstrip("』")
                        t = t.split("』")
                        for k in range(len(t)):
                            t[k] = t[k].split("曲名：『")
                            s.append(t[k][1]) #曲名
                            a.append(t[k][0]) #アーティスト
                            g.append(df_recotai_all.columns.values[j])
                            y.append(i+1)
                else: #「曲名：」表記じゃない
                    if t.startswith("「"):
                        t = t.replace("◆", "")
                        t = t.replace("曲名", "")
                        t = t.lstrip("「")
                        t = t.split("「")
                        for k in range(len(t)):
                            t[k] = t[k].split("」")
                            s.append(t[k][0])
                            a.append(t[k][1])
                            g.append(df_recotai_all.columns.values[j])
                            y.append(i+1)
                    else:
                        t = t.replace("◆", "")
                        t = t.replace("曲名", "")
                        t = t.rstrip("」")
                        t = t.split("」")
                        for k in range(len(t)):
                            t[k] = t[k].split("「")
                            s.append(t[k][1])
                            a.append(t[k][0]) 
                            g.append(df_recotai_all.columns.values[j])
                            y.append(i+1)
    songs.append(s)
    artis.append(a)
    year.append(y)
    genre.append(g)
    
#平坦化
songs = list(itertools.chain.from_iterable(songs))
artis = list(itertools.chain.from_iterable(artis))
year = list(itertools.chain.from_iterable(year))
genre = list(itertools.chain.from_iterable(genre))

In [15]:
new_a = []
for a in artis:
    if "歌手：" in a:
        a = a.split("歌手：")
        a = a[1]
    elif "歌唱：" in a:
        a = a.split("歌唱：")
        a = a[1]
    if "作詩" in a:
        a = a.split("作詩")
        a = a[0]
    if "作曲" in a:
        a = a.split("作曲")
        a = a[0]
    if "プロデューサー" in a:
        a = a.split("プロデューサー")
        a = a[0]
    a = a.lstrip(" ") #先頭の空白削除
    flag = 0
    for i in alpha:
        if i in a:
            flag = 1
            break
    if flag == 0:
        a = a.replace(" ","")        
    new_a.append(a)

In [17]:
#まとめ
df_all = pd.DataFrame(list(zip(year,genre,songs,new_a)), columns = ["回数","賞","曲","歌手"])

li_2020 = [
    [62, "レコード大賞", "炎","LiSA"],
    [62, "優秀作品賞", "I'm Here", "三浦大知"],
    [62, "優秀作品賞","愛をください～Don’t you cry~", "純烈"],
    [62, "優秀作品賞","足跡", "Little Glee Monster"],
    [62, "優秀作品賞","香水", "瑛人"],
    [62, "優秀作品賞","世界中の隣人よ", "乃木坂46"],
    [62, "優秀作品賞","猫", "DISH//"],
    [62, "優秀作品賞","離れていても", "AKB48"],
    [62, "優秀作品賞","母", "氷川きよし"],
    [62, "優秀作品賞","Fantasista～ファンタジスタ～", "DA PUMP"],
    [62, "優秀作品賞","炎", "LiSA"],
    
    [63, "レコード大賞", "CITRUS", "Da-iCE"],
    [63, "優秀作品賞", "明け星", "LiSA"],
    [63, "優秀作品賞", "君がそばにいるから", "純烈"],
    [63, "優秀作品賞", "ごめんねFingers crossed", "乃木坂46"],
    [63, "優秀作品賞", "CITRUS", "Da-iCE"],
    [63, "優秀作品賞", "Take a picture", "NiziU"],
    [63, "優秀作品賞", "Dream on the street", "DA PUMP"],
    [63, "優秀作品賞", "根も葉もRumor", "AKB48"],
    [63, "優秀作品賞", "Backwards", "三浦大知"],
    [63, "優秀作品賞", "Happy!", "氷川きよし"],
    [63, "優秀作品賞", "勿忘", "Awesome City Club"],
]
df2020 = pd.DataFrame(li_2020, columns = ["回数", "賞", "曲", "歌手"])

df_all = pd.DataFrame(pd.concat([df_all,df2020]))

df_all.to_csv("df_matome_1.csv", index=False)
df_all

,回数,賞,曲,歌手
0,1,大賞,黒い花びら,水原弘
1,1,歌唱賞,夜霧に消えたチャコ,フランク永井
2,1,童謡賞,やさしい和尚さん,石井亀次郎とキングホウズキ会
3,2,大賞,誰よりも君を愛す,松尾和子／和田弘とマヒナ・スターズ
4,2,歌唱賞,哀愁波止場,美空ひばり
...,...,...,...,...
17,63,優秀作品賞,Dream on the street,DA PUMP
18,63,優秀作品賞,根も葉もRumor,AKB48
19,63,優秀作品賞,Backwards,三浦大知
20,63,優秀作品賞,Happy!,氷川きよし


In [10]:
#歌ネット上のアーティストIDもってきてCSV書き出し、あまり動かしちゃいけない、長いから

artists_id = []
artists_name = []

def get_tag(html, tag):
    soup = BeautifulSoup(str(html), 'html.parser')
    tag = soup.find_all(tag)
    return tag

for i in range(0,45):
    #あ～を
    url = f"https://www.uta-net.com/name_list/{i}/"
    
    html = requests.get(url)
    html.encoding = html.apparent_encoding #文字化け防止の行
    html = html.text #テキスト表示
    html = BeautifulSoup(str(html), 'html.parser') #きれいに

    for dl in get_tag(html,"div"):
        for dd in get_tag(dl,"dd"):
            for a in get_tag(dd,"a"):
                a = str(a)
                a = a.lstrip('<a href="/artist/')
                if a[-2] == "a": #末尾のaが消えちゃうからこうしてる
                    a = a.rstrip('/a>').rstrip("<")
                else:
                    a = a.rstrip('</a>')
                a = a.split('/">')
                if not "ps" in a[0]:
                    if not "user" in a[0]:
                        artists_id.append(a[0])
                        artists_name.append(a[1])
                        

#IDと名前対応させてデータフレーム化                        
df_artists_id = pd.DataFrame(list(zip(artists_id,artists_name)), columns = ["ID","artist"])
df_artists_id.to_csv("artists_ids.csv", index = False)

In [2]:
#辞書作る(アーティストとID)
df_artists = pd.read_csv("artists_ids.csv")
artists_id = df_artists["ID"]
artists_name = df_artists["artist"]

artists_dic = {key: val for key, val in zip(artists_name, artists_id)}

In [3]:
#試しに
artists_dic["Awesome City Club"]

18088

In [4]:
#関数作っとく
def ch_a(a,b):
    df_all["歌手"][i] = df_all["歌手"][i].replace(a,b)
def ch_s(a,b):
    df_all["曲"][i] = df_all["曲"][i].replace(a,b)

In [5]:
df_all = pd.read_csv("df_matome_1.csv")

In [23]:
#idが見つからない歌手を表示
for i in range(len(df_all)):
    if df_all["歌手"][i] not in artists_dic:
        print(df_all["曲"][i], end = "/")
        print(df_all["歌手"][i])

やさしい和尚さん/石井亀次郎とキングホウズキ会
ゆうらんバス/水上房子／キング小鳩会
団地のお嬢さん/山中みゆき
白い花のブルース/平野こうじ
むすめ三度笠/蔵野たつ美
マーチング・マーチ/天地総子／音羽ゆりかご会
うたう足の歌/杉並合唱団
ペケの歌/長谷川よしみ
うまれたきょうだい１１人/スリー・バブルス
ムーミンのテーマ/玉川さきこ
ママと僕の四季/大鹿太
雪ごもり/芦川よしみ
ABコンプレックス/相川恵理
89番目の星座/伸村知夏
THE 8TH OF ACE/米米CLUB谷村新司


In [22]:
for i in range(len(df_all)):
    #ch_a("石井亀次郎とキングホウズキ会", "")
    ch_a("松尾和子／和田弘とマヒナ・スターズ", "松尾和子＆和田弘とマヒナスターズ")
    #ch_a("水上房子／キング小鳩会", "")
    #ch_a("山中みゆき", "")
    #ch_a("平野こうじ", "")
    #ch_a("蔵野たつ美", "")
    ch_a("橋幸夫/吉永小百合", "橋幸夫＆吉永小百合")
    ch_a("三橋三智也", "三橋美智也")
    #ch_a("天地総子／音羽ゆりかご会", "")
    #ch_a("杉並合唱団", "")
    ch_a("東京ロマンチカ", "鶴岡雅義と東京ロマンチカ")
    #ch_a("長谷川よしみ", "")
    #ch_a("スリー・バブルス", "")
    ch_a("内山田洋とクールファイブ", "内山田洋とクール・ファイブ")
    #ch_a("玉川さきこ", "")
    ch_a("野村真樹", "野村将希")
    ch_a("安倍律子", "安倍里葎子")
    ch_a("ソルティ・シュガー", "ソルティー・シュガー")
    ch_a("にしきのあきら", "錦野旦")
    ch_a("杉並児童合唱団・金森勢", "杉並児童合唱団・お兄さん(金森勢)")
    ch_a("浅丘めぐみ", "麻丘めぐみ")
    ch_a("西条秀樹", "西城秀樹")
    #ch_a("大鹿太", "")
    ch_a("フィンガー５", "フィンガー5")
    #ch_a("芦川よしみ", "")
    ch_a("ピンクレディー", "ピンク・レディー")
    ch_a("ラッツ＆スター", "RATS＆STAR")
    ch_a("サリー（SALLY）", "SALLY")
    ch_a("本田美奈子", "本田美奈子.")
    ch_a("BOØWY", "BOφWY")
    #ch_a("相川恵理", "")
    #ch_a("伸村知夏", "")
    ch_a("BUKFU-SLUMP", "爆風スランプ")
    ch_a("CHAGE & ASKA", "CHAGE and ASKA")
    ch_a("米 米 CLUB", "米米CLUB")
    ch_a("ドリームズ・カム・トゥルー", "DREAMS COME TRUE")
    ch_a("大月みや子", "大月みやこ")
    ch_a("J-WALK", "JAYWALK")
    ch_a("国武万理", "国武万里")
    ch_a("シュー・ビンセイ", "周冰倩")
    ch_a("trf", "TRF")
    #ch_a("米米CLUB谷村新司", "")
    ch_a("MIYA&YAMI", "MIYA・YAMI")
    ch_a("篠原涼子with t.komuro", "篠原涼子 with t.komuro")
    ch_a("try", "TRF")
    ch_a("藤谷美和子／大内義昭", "藤谷美和子・大内義昭")
    ch_a("シャ乱Ｑ", "シャ乱Q")
    ch_a("安室奈美恵withスーパーモンキーズ", "安室奈美恵 with SUPER MONKEY'S")
    ch_a("Puffy", "PUFFY")
    ch_a("オーロラ輝子", "オーロラ輝子(河合美智子)")
    ch_a("しじみとさざえ", "Mi-Ke")
    ch_a("GRAY", "GLAY")
    ch_a("kiroro", "Kiroro")
    ch_a("L’Arc-en-Ciel", "L'Arc～en～Ciel")
    ch_a("１９", "19")
    ch_a("速水けんたろう・茂森あゆみ", "茂森あゆみ・速水けんたろう")
    ch_a("Something Else", "Something ELse")
    ch_a("mihimaruGT", "mihimaru GT")
    ch_a("青山テルマ feat. Soulja", "青山テルマ feat.SoulJa")
    ch_a("GIRL NEXT DOOR", "girl next door")
    ch_a("BIG BANG", "BIGBANG")
    ch_a("w-ins. x G-DRAGON(BIGBANG)", "w-inds.×G-DRAGON(BIG BANG)")
    ch_a("水森かおりJUJU", "JUJU")
    
    ch_a("三代目 J Soul Brothers", "三代目 J SOUL BROTHERS from EXILE TRIBE")
    ch_a("三代目 Ｊ Soul Brothers from EXILE TRIBE", "三代目 J SOUL BROTHERS from EXILE TRIBE")
    ch_a("三代目 J SOUL BROTHERS from EXILE TRIBE from EXILE TRIBE", "三代目 J SOUL BROTHERS from EXILE TRIBE")

    ch_a("Fairies", "フェアリーズ")
    ch_a("AKB46", "AKB48")
    ch_a("三浦大和", "三浦大知")
    ch_a("ボニージャックス", "童謡・唱歌")
    ch_a("田代美代子", "和田弘とマヒナスターズ＆田代美代子")

C:\Users\HAKUTA~1.YUK\AppData\Local\Temp/ipykernel_18020/2560360093.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all["歌手"][i] = df_all["歌手"][i].replace(a,b)


In [24]:
df_all.to_csv("df_matome_2.csv", index=False)
df_all

,回数,賞,曲,歌手
0,1,大賞,黒い花びら,水原弘
1,1,歌唱賞,夜霧に消えたチャコ,フランク永井
2,1,童謡賞,やさしい和尚さん,石井亀次郎とキングホウズキ会
3,2,大賞,誰よりも君を愛す,松尾和子＆和田弘とマヒナスターズ
4,2,歌唱賞,哀愁波止場,美空ひばり
...,...,...,...,...
800,63,優秀作品賞,Dream on the street,DA PUMP
801,63,優秀作品賞,根も葉もRumor,AKB48
802,63,優秀作品賞,Backwards,三浦大知
803,63,優秀作品賞,Happy!,氷川きよし


In [6]:
df_all = pd.read_csv("df_matome_2.csv")
df_all

,回数,賞,曲,歌手
0,1,大賞,黒い花びら,水原弘
1,1,歌唱賞,夜霧に消えたチャコ,フランク永井
2,1,童謡賞,やさしい和尚さん,石井亀次郎とキングホウズキ会
3,2,大賞,誰よりも君を愛す,松尾和子＆和田弘とマヒナスターズ
4,2,歌唱賞,哀愁波止場,美空ひばり
...,...,...,...,...
800,63,優秀作品賞,Dream on the street,DA PUMP
801,63,優秀作品賞,根も葉もRumor,AKB48
802,63,優秀作品賞,Backwards,三浦大知
803,63,優秀作品賞,Happy!,氷川きよし


In [8]:
#アーティストの曲の歌ネット上ID持ってくる&データフレーム化、滅多に動かすものじゃない
song = []

jushousha = set(df_all["歌手"])
jushousha_plus = ["三沢あけみ・和田弘とマヒナスターズ", "島本須美", "橋幸夫・若草児童合唱団", "都はるみ＆岡千秋", "石原裕次郎・川中美幸"]
jushousha = list(jushousha) + jushousha_plus
jushousha = set(jushousha)

def get_tag(html, tag):
    soup = BeautifulSoup(str(html), 'html.parser')
    tag = soup.find_all(tag)
    return tag

for artist in jushousha:
    #各アーティストのページ数表示
    #print(artist, end = "：")
    if artist in artists_dic:
        a_id = artists_dic[f"{artist}"]
        url = f"https://www.uta-net.com/artist/{a_id}/"

        #何ページあるか確認
        html = requests.get(url)
        html.encoding = html.apparent_encoding #文字化け防止の行
        html = html.text #テキスト表示
        html = BeautifulSoup(str(html), 'html.parser') #きれいに

        page = html.find_all('span', class_="pa")
        page = str(page)
        page = page.lstrip('[<span class="pa">全')
        page = page.rstrip('ページ中　1ページを表示</span>]')
        if page == "":
            page = 1
        page = int(page)
        #print(page)

        for p in range(page):
            #print(p+1)
            url_p = f"https://www.uta-net.com/artist/{a_id}/0/{p+1}/"
            html = requests.get(url_p)
            html.encoding = html.apparent_encoding #文字化け防止の行
            html = html.text #テキスト表示
            html = BeautifulSoup(str(html), 'html.parser') #きれいに

            for div in get_tag(html,"div"):
                for td in get_tag(div,"td"):
                    for a in get_tag(td,"a"):
                        a = str(a)
                        a = a.replace("&amp;", "&") #HTMLエンコード対処
                        a = a.lstrip('<a href="/song/')
                        if a[-2] == "a": #末尾のaが消えちゃうからこうしてる
                            a = a.rstrip('/a>').rstrip("<")
                        else:
                            a = a.rstrip('</a>')
                        a = a.split('/">')
                        if not "ps" in a[0]:
                            if not "user" in a[0]:
                                #曲名と曲ID分割したところにアーティスト名加える
                                a[1] = a[1]+"_by_"+artist
                                song.append(a)

    df_song_by_recoartists = pd.DataFrame(song, columns = ["ID", "song"])

In [9]:
#重複行削除
df_song_by_recoartists = df_song_by_recoartists[~df_song_by_recoartists.duplicated()]
df_song_by_recoartists.to_csv("songs_ids_1.csv", index = False)
df_song_by_recoartists

,ID,song
0,81616,哀愁トラベラー_by_渡辺真知子
1,151575,愛情パズル_by_渡辺真知子
2,125127,愛することだけすればよかった_by_渡辺真知子
3,192410,愛の讃歌_by_渡辺真知子
4,157008,青空に誓って_by_渡辺真知子
...,...,...
105175,6787,WAKE UP!_by_BaBe
105176,42611,Give Me Up_by_BaBe
105177,2114,Somebody Loves You ～明日の恋人～_by_BaBe
105178,6788,TONIGHT!_by_BaBe


In [10]:
#辞書つくる(曲(レコ大受賞アーティスト限定)とID)

#ここから！
df_all = pd.read_csv("df_matome_2.csv")
df_song_by_recoartists = pd.read_csv("songs_ids_1.csv")

song_name = df_song_by_recoartists["song"]
song_name = song_name.values.tolist()
song_num = df_song_by_recoartists["ID"]
song_num = song_num.values.tolist()

song_dic = {key: val for key, val in zip(song_name, song_num)}

In [13]:
df_all = pd.read_csv("df_matome_2.csv")

#曲名修正
for i in range(len(df_all)):
    ch_s("小さい秋みつけた", "ちいさい秋みつけた")
    #ch_s("島のブルース_by_三沢あけみ", "")
    #ch_s("私も流れの渡り鳥_by_三沢あけみ", "")
    #ch_s("ワンツースリーゴー_by_音羽ゆりかご会", "")
    ch_s("17才のこの胸に", "十七才のこの胸に")
    ch_s("ワンレイニーナイト・イン・トーキョー", "ワン・レイニー・ナイト・イン・トーキョー")
    #ch_s("マーチング・マーチ_by_天地総子／音羽ゆりかご会", "")
    ch_s("おばけのQ太郎", "オバケのQ太郎")
    #ch_s("うたう足の歌_by_杉並合唱団", "")
    #ch_s("ペケの歌_by_長谷川よしみ", "")
    ch_s("ひとり寝の子守歌", "ひとり寝の子守唄")
    #ch_s("うまれたきょうだい１１人_by_スリー・バブルス", "")
    ch_s("365歩のマーチ", "三百六十五歩のマーチ")
    ch_s("波止場おんなのブルース", "波止場女のブルース")
    #ch_s("ムーミンのテーマ_by_玉川さきこ", "")
    ch_s("走れコータロー", "走れコウタロー")
    ch_s("よこはまたそがれ", "よこはま・たそがれ")
    #ch_s("じんじん_by_ひばり児童合唱団", "")
    ch_s("私の城下町", "わたしの城下町")
    #ch_s("子連れ狼_by_橋幸夫", "")
    #ch_s("ママと僕の四季_by_大鹿太", "")
    #ch_s("恋はダンダン_by_浅野ゆう子", "")
    #ch_s("雪ごもり_by_芦川よしみ", "")
    ch_s("思い出ぼろぼろ", "想い出ぼろぼろ")
    ch_s("秋桜（コスモス）", "秋桜")
    ch_s("津軽海峡冬景色", "津軽海峡・冬景色")
    ch_s("あずさ２号", "あずさ2号")
    ch_s("Lui-Lui（ルイ・ルイ）", "Lui-Lui")
    ch_s("ウォンテッド", "ウォンテッド(指名手配)")
    ch_s("ＵＦＯ", "UFO")
    ch_s("LOVE（抱きしめたい）", "LOVE ～抱きしめたい～")
    ch_s("グッド・ラック  Good Luck", "グッド・ラック")
    ch_s("ブルースカイブルー", "ブルースカイ ブルー")
    ch_s("大坂しぐれ", "大阪しぐれ")
    ch_s("別れても好きなひと", "別れても好きな人")
    ch_s("ルビーの指輪", "ルビーの指環")
    #ch_s("てれてZinZin_by_竹本孝之", "")
    ch_s("ス・ト・リ・ッ・パ・ー", "ス・ト・リ・ッ・パー")
    ch_s("センチメンタルガール", "センチメンタル・ガール")
    ch_s("100%…Soかもね", "100％…SOかもね！")
    ch_s("６番目のユ・ウ・ウ・ツ", "6番目のユ・ウ・ウ・ツ")
    #ch_s("恋あなたしだい_by_岩井小百合", "")
    #ch_s("ダンシング・レディ_by_大沢逸美", "")
    ch_s("きまぐれONE WAY BOY", "気まぐれONE WAY BOY")
    ch_s("ミッド・ナイト・ステーション", "ミッドナイト ステーション")
    #ch_s("浪花恋しぐれ_by_都はるみ", "")
    ch_s("サマーアイズ", "SUMMER EYES")
    ch_s("恋、はじめまして", "―Dreaming Girl― 恋、はじめまして")
    ch_s("恋 はじめまして", "―Dreaming Girl― 恋、はじめまして")
    ch_s("北の蛍", "北の螢")
    ch_s("もしかしてPART II", "もしかして PART II")
    ch_s("Temptation", "Temptation(誘惑)")
    ch_s("恋におちて", "恋におちて-Fall in Love-")
    #ch_s("C_by_中山美穂", "")
    #ch_s("さよならと言われて_by_松本典子", "")
    #ch_s("Ｃ_by_中山美穂", "")
    ch_s("デビュー", "デビュー ～Fly Me To Love～")
    ch_s("DESIRE", "DESIRE -情熱-")
    ch_s("CYA-CHA-CYA", "CHA-CHA-CHA")
    ch_s("わたし、ドリーミング", "わたし・ドリーミング")
    #ch_s("夢飛行_by_真璃子", "")
    ch_s("浪花杯", "浪花盃")
    ch_s("ノ・レ・な・い Teen—age", "ノ・レ・な・い Teen-age")
    ch_s("キミはとんどくらい", "キミはどんとくらい")
    ch_s("I Don’T know", "I Don't Know!")
    ch_s("Marionette", "Marionette ～マリオネット～")
    #ch_s("ABコンプレックス_by_相川恵理", "")
    #ch_s("89番目の星座_by_伸村知夏", "")
    ch_s("JUST ONE MORE KISS TO SEARCHI", "JUST ONE MORE KISS")
    ch_s("王将一代小春しぐれ", "王将一代 小春しぐれ")
    ch_s("あぁ、グッと", "ああ、グッと")
    ch_s("ストレンジャーTonight", "ストレンジャーtonight")
    ch_s("BEYOND THE TIME", "BEYOND THE TIME(メビウスの宇宙を越えて)")
    ch_s("You’re My Only SHINEN’ STAR", "You're My Only Shinin' Star")
    #ch_s("夢だけみてる_by_川越美和", "")
    ch_s("男同士", "男同志")
    ch_s("ユア・マイ・ライフ", "ユア・マイ・ライフ(YOU'RE MY LIFE)")
    ch_s("リゾ・ラバ", "リゾ・ラバ -resort lovers-")
    #ch_s("赤い花束_by_中嶋美智代", "")
    ch_s("どんなときも…", "どんなときも。")
    #ch_s("今度私どこか連れていって下さいよ_by_加藤紀子", "")
    ch_s("You’re the only…", "You're the Only…")
    ch_s("Get Along Together", "Get Along Together -愛を贈りたいから-")
    #ch_s("THE 8TH OF ACE_by_米米CLUB谷村新司", "")
    ch_s("憧夢〜風に向かって", "憧夢 ～風に向かって～")
    #ch_s("逢えるじゃないかまたあした_by_川中美幸", "")
    ch_s("花のように 鳥のように", "花のように鳥のように")
    ch_s("survivaldAnce〜no no cry more", "survival dAnce ～no no cry more～")
    ch_s("OVERNIGHT SENSATION 〜時代はあなたに委ねてる〜", "Overnight Sensation ～時代はあなたに委ねてる～")
    ch_s("OVERNIGHT SENSATION〜時代はあなたに委ねている〜", "Overnight Sensation ～時代はあなたに委ねてる～")
    ch_s("シーソーゲーム〜勇敢な恋の歌〜", "シーソーゲーム ～勇敢な恋の歌～")
    ch_s("TRY ME〜私を信じて〜", "TRY ME ～私を信じて～")
    ch_s("Don’t wanna cry", "Don't wanna cry")
    ch_s("バンザイ〜好きでよかった〜", "バンザイ ～好きでよかった～")    
    ch_s("I’m Proud", "I'm proud")
    ch_s("蛍の提灯", "螢の提灯")
    ch_s("WHITE LOVE", "White Love")
    ch_s("Give me Shake", "Give me a Shake")
    ch_s("Wanna Be A Dreammaker", "wanna Be A Dreammaker")
    ch_s("Winter, again", "Winter,again")
    ch_s("君影草", "君影草 -すずらん-")
    ch_s("だんご３兄弟", "だんご3兄弟")
    ch_s("Boys&Girls", "Boys & Girls")
    ch_s("あなたのキスを数えましょう 〜You were mine〜", "あなたのキスを数えましょう～You were mine～")
    ch_s("if…", "if...")
    ch_s("Steppin’and Shakin'", "Steppin' and Shakin'")
    ch_s("おんなの一生〜汗の花〜", "おんなの一生～汗の花～")
    ch_s("DOUBLE", "Double")
    ch_s("童神〜ヤマトグチ〜", "童神～ヤマトグチ～")
    ch_s("時の描片 〜トキノカケラ〜", "時の描片～トキノカケラ～")
    ch_s("PEACHI", "PEACH")
    ch_s("蕾（つぼみ）", "蕾")
    ch_s("江戸の手鞠唄II", "江戸の手毬唄II")
    ch_s("やぁ無情", "やぁ 無情")
    ch_s("手紙〜親愛なる子供たちへ〜", "手紙～親愛なる子供たちへ～")
    ch_s("Lick me♥", "Lick me")
    ch_s("Rain Is Fallin'", "Rain is Fallin'")
    ch_s("虹色バイヨン", "虹色のバイヨン")
    ch_s("Why? (Keep Your Head Down)", "Why?(Keep Your Head Down)")
    ch_s("真夏のSound good!", "真夏のSounds good !")
    ch_s("GO FOR IT!!", "GO FOR IT !!")
    ch_s("777 〜We can sing a song! 〜", "777 ～We can sing a song!～")
    ch_s("EXILE PRIDE〜こんな世界を愛するため〜", "EXILE PRIDE ～こんな世界を愛するため～")
    ch_s("EXILE PRIDE 〜こんな世界を愛するために〜", "EXILE PRIDE ～こんな世界を愛するため～")
    ch_s("満天の瞳", "満天の瞳(ほし)")
    ch_s("PPAP (ペンパイナッポーアッポーペン)", "PPAP(ペンパイナッポーアッポーペン)")
    ch_s("U.S.A", "U.S.A.")
    ch_s("P.A.R.T.Y.〜ユニバース・フェスティバル〜", "P.A.R.T.Y. ～ユニバース・フェスティバル～")
    ch_s("愛をください～Don’t you cry~", "愛をください～Don't you cry～")
    
    ch_s("ものまね！想い出の九十九里浜", "想い出の九十九里浜")
    ch_s("Ｒ.Ｙ.Ｕ.Ｓ.Ｅ.Ｉ", "R.Y.U.S.E.I.")
    ch_s("R.Y.U.S.E.I..", "R.Y.U.S.E.I.")
    ch_s("Daling", "Darling")
    ch_s("ありがとうForever…", "ありがとうForever...")
    
    ch_s("愛よ愛よ[かなよかなよ]", "愛よ愛よ")
    ch_s("氷見[ひみ]の雪", "氷見の雪")
    ch_s("胸焦がす", "胸焦がす...")
    ch_s("沙[すな]の川", "沙の川")

C:\Users\HAKUTA~1.YUK\AppData\Local\Temp/ipykernel_7956/2560360093.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all["曲"][i] = df_all["曲"][i].replace(a,b)


In [14]:
song_id = []
song_by = []

for i in range(len(df_all)):
    sb = df_all["曲"][i]+"_by_"+df_all["歌手"][i]
    song_by.append(sb)

for s in song_by:
    s = str(s)
    if s in song_dic:
        song_id.append(song_dic[s])
    else:
        song_id.append("No ID!")
        print(f'ch_s("{s}", "")')

df_matome_all = pd.DataFrame(list(zip(song_id,song_by)), columns = ["ID", "song_by"])
df_matome_all

ch_s("やさしい和尚さん_by_石井亀次郎とキングホウズキ会", "")
ch_s("ゆうらんバス_by_水上房子／キング小鳩会", "")
ch_s("団地のお嬢さん_by_山中みゆき", "")
ch_s("白い花のブルース_by_平野こうじ", "")
ch_s("むすめ三度笠_by_蔵野たつ美", "")
ch_s("島のブルース_by_三沢あけみ", "")
ch_s("私も流れの渡り鳥_by_三沢あけみ", "")
ch_s("ワンツースリーゴー_by_音羽ゆりかご会", "")
ch_s("マーチング・マーチ_by_天地総子／音羽ゆりかご会", "")
ch_s("うたう足の歌_by_杉並合唱団", "")
ch_s("ペケの歌_by_長谷川よしみ", "")
ch_s("うまれたきょうだい１１人_by_スリー・バブルス", "")
ch_s("ムーミンのテーマ_by_玉川さきこ", "")
ch_s("じんじん_by_ひばり児童合唱団", "")
ch_s("子連れ狼_by_橋幸夫", "")
ch_s("ママと僕の四季_by_大鹿太", "")
ch_s("恋はダンダン_by_浅野ゆう子", "")
ch_s("雪ごもり_by_芦川よしみ", "")
ch_s("てれてZinZin_by_竹本孝之", "")
ch_s("恋あなたしだい_by_岩井小百合", "")
ch_s("ダンシング・レディ_by_大沢逸美", "")
ch_s("浪花恋しぐれ_by_都はるみ", "")
ch_s("C_by_中山美穂", "")
ch_s("さよならと言われて_by_松本典子", "")
ch_s("Ｃ_by_中山美穂", "")
ch_s("夢飛行_by_真璃子", "")
ch_s("ABコンプレックス_by_相川恵理", "")
ch_s("89番目の星座_by_伸村知夏", "")
ch_s("夢だけみてる_by_川越美和", "")
ch_s("赤い花束_by_中嶋美智代", "")
ch_s("今度私どこか連れていって下さいよ_by_加藤紀子", "")
ch_s("THE 8TH OF ACE_by_米米CLUB谷村新司", "")
ch_s("逢えるじゃないかまたあした_by_川中美幸", "")


,ID,song_by
0,181713,黒い花びら_by_水原弘
1,42705,夜霧に消えたチャコ_by_フランク永井
2,No ID!,やさしい和尚さん_by_石井亀次郎とキングホウズキ会
3,13077,誰よりも君を愛す_by_松尾和子＆和田弘とマヒナスターズ
4,14764,哀愁波止場_by_美空ひばり
...,...,...
800,298572,Dream on the street_by_DA PUMP
801,308342,根も葉もRumor_by_AKB48
802,299323,Backwards_by_三浦大知
803,307477,Happy!_by_氷川きよし


In [15]:
#曲と曲IDを一致させる

song_id = []
song_by = []

for i in range(len(df_all)):
    sb = df_all["曲"][i]+"_by_"+df_all["歌手"][i]
    song_by.append(sb)
    
#修正
def ch_sb(a,b):
    song_by[i] = song_by[i].replace(a,b)
for i in range(len(song_by)):
    ch_sb("島のブルース_by_三沢あけみ", "島のブルース_by_三沢あけみ・和田弘とマヒナスターズ")
    ch_sb("マーチング・マーチ_by_天地総子／音羽ゆりかご会", "マーチングマーチ_by_童謡・唱歌")
    ch_sb("ムーミンのテーマ_by_玉川さきこ", "ムーミンのテーマ_by_島本須美")
    ch_sb("子連れ狼_by_橋幸夫", "子連れ狼_by_橋幸夫・若草児童合唱団")
    ch_sb("浪花恋しぐれ_by_都はるみ", "浪花恋しぐれ_by_都はるみ＆岡千秋")
    ch_sb("C_by_中山美穂", "「C」_by_中山美穂")
    ch_sb("Ｃ_by_中山美穂", "「C」_by_中山美穂")
    ch_sb("逢えるじゃないかまたあした_by_川中美幸", "逢えるじゃないかまたあした_by_石原裕次郎・川中美幸")
    ch_sb("哀愁港_by_五木ひろし", "哀愁港_by_田川寿美")

for s in song_by:
    s = str(s)
    if s in song_dic:
        song_id.append(song_dic[s])
    else:
        song_id.append("No ID!")
        print(f'ch_sb("{s}", "")')

df_matome_all = pd.DataFrame(list(zip(df_all["回数"],df_all["賞"], song_id, song_by)), columns = ["回数", "賞", "ID", "song_by"])
df_matome_all.to_csv("df_matome_3.csv", index = False)
df_matome_all

ch_sb("やさしい和尚さん_by_石井亀次郎とキングホウズキ会", "")
ch_sb("ゆうらんバス_by_水上房子／キング小鳩会", "")
ch_sb("団地のお嬢さん_by_山中みゆき", "")
ch_sb("白い花のブルース_by_平野こうじ", "")
ch_sb("むすめ三度笠_by_蔵野たつ美", "")
ch_sb("私も流れの渡り鳥_by_三沢あけみ", "")
ch_sb("ワンツースリーゴー_by_音羽ゆりかご会", "")
ch_sb("うたう足の歌_by_杉並合唱団", "")
ch_sb("ペケの歌_by_長谷川よしみ", "")
ch_sb("うまれたきょうだい１１人_by_スリー・バブルス", "")
ch_sb("じんじん_by_ひばり児童合唱団", "")
ch_sb("ママと僕の四季_by_大鹿太", "")
ch_sb("恋はダンダン_by_浅野ゆう子", "")
ch_sb("雪ごもり_by_芦川よしみ", "")
ch_sb("てれてZinZin_by_竹本孝之", "")
ch_sb("恋あなたしだい_by_岩井小百合", "")
ch_sb("ダンシング・レディ_by_大沢逸美", "")
ch_sb("さよならと言われて_by_松本典子", "")
ch_sb("夢飛行_by_真璃子", "")
ch_sb("ABコンプレックス_by_相川恵理", "")
ch_sb("89番目の星座_by_伸村知夏", "")
ch_sb("夢だけみてる_by_川越美和", "")
ch_sb("赤い花束_by_中嶋美智代", "")
ch_sb("今度私どこか連れていって下さいよ_by_加藤紀子", "")
ch_sb("THE 8TH OF ACE_by_米米CLUB谷村新司", "")


,回数,賞,ID,song_by
0,1,大賞,181713,黒い花びら_by_水原弘
1,1,歌唱賞,42705,夜霧に消えたチャコ_by_フランク永井
2,1,童謡賞,No ID!,やさしい和尚さん_by_石井亀次郎とキングホウズキ会
3,2,大賞,13077,誰よりも君を愛す_by_松尾和子＆和田弘とマヒナスターズ
4,2,歌唱賞,14764,哀愁波止場_by_美空ひばり
...,...,...,...,...
800,63,優秀作品賞,298572,Dream on the street_by_DA PUMP
801,63,優秀作品賞,308342,根も葉もRumor_by_AKB48
802,63,優秀作品賞,299323,Backwards_by_三浦大知
803,63,優秀作品賞,307477,Happy!_by_氷川きよし


In [16]:
#歌詞持ってくる

lyrics = []

def get_id(html, id):
    soup = BeautifulSoup(str(html), 'html.parser')
    html_id = soup.select(id)
    return html_id

for i in range(len(df_matome_all)):
    if df_matome_all["ID"][i] != "No ID!":
        url = f'https://www.uta-net.com/song/{df_matome_all["ID"][i]}/'

        html = requests.get(url)
        html.encoding = html.apparent_encoding #文字化け防止の行
        html = html.text #テキスト表示
        html = BeautifulSoup(str(html), 'html.parser') #きれいに

        l = get_id(html,"#kashi_area")
        l = BeautifulSoup(str(l), 'html.parser')
        l = l.get_text(separator=" ").strip()
        l = l.replace("\u3000", " ")
        l = l.lstrip("[ ")
        l = l.rstrip(" ]")
        lyrics.append(l)
    else:
        lyrics.append("Not Found...")

In [17]:
#データフレーム化
li_song = df_matome_all["song_by"]
df_lyrics = pd.DataFrame(list(zip(li_song, lyrics)), columns = ["song_by", "歌詞"])
df_all = pd.read_csv("df_matome_2.csv")

#まとめる
df_recotai_matome2 = df_matome_all.merge(df_lyrics)
df_recotai_matome2 = df_recotai_matome2[~df_recotai_matome2.duplicated()] #重複行削除
df_recotai_matome2.to_csv("df_matome_4.csv", index = False)
df_recotai_matome2

,回数,賞,ID,song_by,歌詞
0,1,大賞,181713,黒い花びら_by_水原弘,黒い花びら 静かに散った あの人は帰らぬ 遠い夢 俺は知ってる 恋の悲しさ 恋の苦しさ だか...
1,1,歌唱賞,42705,夜霧に消えたチャコ_by_フランク永井,俺のこころを 知りながら なんでだまって 消えたんだ チャコ チャコ 酒場に咲いた 花だけど...
2,1,童謡賞,No ID!,やさしい和尚さん_by_石井亀次郎とキングホウズキ会,Not Found...
3,2,大賞,13077,誰よりも君を愛す_by_松尾和子＆和田弘とマヒナスターズ,誰にも云われず たがいに誓った かりそめの恋なら 忘れもしようが ああ 夢ではない ただひと...
4,2,歌唱賞,14764,哀愁波止場_by_美空ひばり,夜の波止場にゃ 誰あれもいない 霧にブイの灯 泣くばかり おどま盆ぎり盆ぎり 盆からさきゃ ...
...,...,...,...,...,...
954,63,優秀作品賞,298572,Dream on the street_by_DA PUMP,指パッチン [Snap your fingers] リズム・マジック [Get up in ...
955,63,優秀作品賞,308342,根も葉もRumor_by_AKB48,Hey！ Whazzup？ Nothing！ Nothing！ Nothing！ Nothi...
956,63,優秀作品賞,299323,Backwards_by_三浦大知,始まりがなければ 終わりだってない 呼ばれ顔上げれば 目にかかる前髪 踏み出しても 変わらな...
957,63,優秀作品賞,307477,Happy!_by_氷川きよし,がんばりすぎて いつも 強がり 見栄張り 眠れない日も 絶えず 笑顔を ふりまいて そんな君...


In [18]:
#手作業
df_recotai_matome2 = pd.read_csv("df_matome_4.csv")

f = open('yasa.txt', 'r', encoding='UTF-8')
yasa = f.read()
f.close()
yasa = yasa.replace("\u3000", " ")
yasa = yasa.replace("\n", " ")
for i in range(len(df_recotai_matome2)):
    if df_recotai_matome2["song_by"][i] == "やさしい和尚さん_by_石井亀次郎とキングホウズキ会":
        df_recotai_matome2["歌詞"][i] = yasa
#https://www.youtube.com/watch?v=JXFvHvGwSu4

f = open('wantu.txt', 'r', encoding='UTF-8')
wantu = f.read()
f.close()
wantu = wantu.replace("\u3000", " ")
wantu = wantu.replace("\n", " ")
for i in range(len(df_recotai_matome2)):
    if df_recotai_matome2["song_by"][i] == "ワンツースリーゴー_by_音羽ゆりかご会":
        df_recotai_matome2["歌詞"][i] = wantu
#https://detail.chiebukuro.yahoo.co.jp/qa/question_detail/q1222365392

f = open('hachiju.txt', 'r', encoding='UTF-8')
hachiju = f.read()
f.close()
hachiju = hachiju.replace("\u3000", " ")
hachiju = hachiju.replace("\n", " ")
for i in range(len(df_recotai_matome2)):
    if df_recotai_matome2["song_by"][i] == "89番目の星座_by_伸村知夏":
        df_recotai_matome2["歌詞"][i] = hachiju
#https://www.youtube.com/watch?v=J-rYhcdEJEQ

f = open('koiana.txt', 'r', encoding='UTF-8')
koiana = f.read()
f.close()
koiana = koiana.replace("\u3000", " ")
koiana = koiana.replace("\n", " ")
for i in range(len(df_recotai_matome2)):
    if df_recotai_matome2["song_by"][i] == "恋あなたしだい_by_岩井小百合":
        df_recotai_matome2["歌詞"][i] = koiana
#https://www.musixmatch.com/ja/lyrics/岩井小百合/恋あなたし・だ・い

f = open('mamaboku.txt', 'r', encoding='UTF-8')
text = f.read()
f.close()
text = text.replace("\u3000", " ")
text = text.replace("\n", " ")
for i in range(len(df_recotai_matome2)):
    if df_recotai_matome2["song_by"][i] == "ママと僕の四季_by_大鹿太":
        df_recotai_matome2["歌詞"][i] = text
        
f = open('terete.txt', 'r', encoding='UTF-8')
text = f.read()
f.close()
text = text.replace("\u3000", " ")
text = text.replace("\n", " ")
for i in range(len(df_recotai_matome2)):
    if df_recotai_matome2["song_by"][i] == "てれてZinZin_by_竹本孝之":
        df_recotai_matome2["歌詞"][i] = text
        
f = open('dandan.txt', 'r', encoding='UTF-8')
text = f.read()
f.close()
text = text.replace("\u3000", " ")
text = text.replace("\n", " ")
for i in range(len(df_recotai_matome2)):
    if df_recotai_matome2["song_by"][i] == "恋はダンダン_by_浅野ゆう子":
        df_recotai_matome2["歌詞"][i] = text    
        
f = open('AB.txt', 'r', encoding='UTF-8')
text = f.read()
f.close()
text = text.replace("\u3000", " ")
text = text.replace("\n", " ")
for i in range(len(df_recotai_matome2)):
    if df_recotai_matome2["song_by"][i] == "ABコンプレックス_by_相川恵理":
        df_recotai_matome2["歌詞"][i] = text    
        
#確認用        
for i in range(len(df_recotai_matome2)):
    if df_recotai_matome2["歌詞"][i] == "Not Found...":
        a = df_recotai_matome2.iloc[i,3]
        print(str(i)+":"+a)
        
df_recotai_matome2.to_csv("df_matome_4.csv", index = False)

5:ゆうらんバス_by_水上房子／キング小鳩会
11:団地のお嬢さん_by_山中みゆき
12:白い花のブルース_by_平野こうじ
14:むすめ三度笠_by_蔵野たつ美
27:私も流れの渡り鳥_by_三沢あけみ
47:うたう足の歌_by_杉並合唱団
54:ペケの歌_by_長谷川よしみ
62:うまれたきょうだい１１人_by_スリー・バブルス
88:じんじん_by_ひばり児童合唱団
156:雪ごもり_by_芦川よしみ
276:ダンシング・レディ_by_大沢逸美
324:さよならと言われて_by_松本典子
342:夢飛行_by_真璃子
400:夢だけみてる_by_川越美和
432:赤い花束_by_中嶋美智代
453:今度私どこか連れていって下さいよ_by_加藤紀子
475:THE 8TH OF ACE_by_米米CLUB谷村新司


C:\Users\HAKUTA~1.YUK\AppData\Local\Temp/ipykernel_7956/1072911020.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_recotai_matome2["歌詞"][i] = yasa
C:\Users\HAKUTA~1.YUK\AppData\Local\Temp/ipykernel_7956/1072911020.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_recotai_matome2["歌詞"][i] = wantu
C:\Users\HAKUTA~1.YUK\AppData\Local\Temp/ipykernel_7956/1072911020.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_recot

In [19]:
#J_Lyricから
df_all = pd.read_csv("df_matome_4.csv")

j_lyric = ["https://j-lyric.net/artist/a053f82/l021320.html",
          "https://j-lyric.net/artist/a00284a/l021328.html",
          "https://j-lyric.net/artist/a0544a4/l02755a.html",
          "https://j-lyric.net/artist/a002e4e/l00b4b4.html",
          "https://j-lyric.net/artist/a0056e2/l02c4de.html",
          "https://j-lyric.net/artist/a000ddb/l02a68a.html",
          "https://j-lyric.net/artist/a000d94/l01bd54.html",
          "https://j-lyric.net/artist/a002442/l046fee.html"]
jl_li = []
for url in j_lyric:    
    jl = []
    html = requests.get(url)
    html.encoding = html.apparent_encoding #文字化け防止の行
    html = html.text #テキスト表示
    soup = BeautifulSoup(str(html), 'html.parser') #きれいに
    for j in soup.find_all("div", class_="cap"):
        text = j.get_text(strip=True)
        if "「" in text:
            text = text.lstrip("「").rstrip("」歌詞")
            text = text.replace("夢だけ見てる", "夢だけみてる")
            jl.append(text)
    for j in soup.find_all("p", class_="sml"):
        text = j.get_text(strip=True)
        if "歌：" in text:
            text = str(text).lstrip("歌：")
            jl.append(text)
    for j in soup.find_all(["p", "div"]):
        text = j.get_text(strip=True)
        if j.name == "p" and j.get('id') == 'Lyric':
            jl.append(text)
    sb = f"{jl[0]}_by_{jl[1]}"
    jl2 = [sb, jl[2]]
    jl_li.append(jl2)
#当てはめる
for i in range(len(jl_li)):
    for j in range(len(df_all)):
        if jl_li[i][0] == df_all["song_by"][j]:
            df_all["歌詞"][j] = jl_li[i][1]
            break
            
#utauashi
url = "https://kashinavi.com/song_view.html?15062"
html = requests.get(url)
html.encoding = html.apparent_encoding #文字化け防止の行
html = html.text #テキスト表示
soup = BeautifulSoup(str(html), 'html.parser') #きれいに
utauashi = ["うたう足の歌_by_杉並合唱団"]
for i in soup.find_all("div", class_="kashi"):
    text = i.get_text(strip=True)
    text = text.replace("\u3000", " ")
    utauashi.append(text)

for j in range(len(df_all)):
    if utauashi[0] == df_all["song_by"][j]:
        df_all["歌詞"][j] = utauashi[1]
        break
        
df_all.to_csv("df_last.csv", index = False)

C:\Users\HAKUTA~1.YUK\AppData\Local\Temp/ipykernel_7956/4284705541.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all["歌詞"][j] = jl_li[i][1]
C:\Users\HAKUTA~1.YUK\AppData\Local\Temp/ipykernel_7956/4284705541.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all["歌詞"][j] = utauashi[1]
